## Main run datacard script

This is the main script for running the X,Y datacard class,
detailed in JIRA SAIV-2284.
To interact with the class, construct it as follows:


##### DC = Datacard(client=CLIENT, data_paths=data_paths, report_conf_dir='.', report_output_dir='.', experiment_dates=EXPERIMENT_DATES)
               
And then run the .generate_datacard() method.

##### DC.generate_datacard()

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
import json
import os
import sys
import pandas as pd
from pathlib import Path
import boto3
from eliot import log_message
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy import text
import pickle
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from PIL import Image

from saiva.model.shared.constants import MODEL_TYPE
from saiva.model.shared.utils import get_client_class
from saiva.model.shared.datacards.feature_label_datacard import Datacard

## Load config

In [ ]:
from saiva.model.shared.constants import LOCAL_TRAINING_CONFIG_PATH
from saiva.training.utils import load_config

config = load_config(LOCAL_TRAINING_CONFIG_PATH)
training_config = config.training_config

In [ ]:
MODEL_TYPE = MODEL_TYPE.lower()
CLIENT = "+".join([config.organization_id for config in training_config.organization_configs])

processed_path = Path('./data/processed')
data_path = Path('./data/raw')

EXPERIMENT_DATES = training_config.training_metadata.experiment_dates
# starting training from day 31 so that cumsum window 2,7,14,30 are all initial correct.
EXPERIMENT_DATES['train_start_date'] = str((pd.to_datetime(EXPERIMENT_DATES['train_start_date']) +  pd.DateOffset(days=30)).date())


In [ ]:
processed_path = Path('/data/processed')
data_path = Path('/data/raw')

In [ ]:
data_paths = {'X_train': processed_path/f'final-train_x_{MODEL_TYPE}.pickle',
              'Y_train': processed_path/f'final-train_target_3_day_{MODEL_TYPE}.pickle',
              'idens_train': processed_path/f'final-train_idens_{MODEL_TYPE}.pickle',
              'X_valid': processed_path/f'final-valid_x_{MODEL_TYPE}.pickle',
              'Y_valid': processed_path/f'final-valid_target_3_day_{MODEL_TYPE}.pickle',
              'idens_valid': processed_path/f'final-valid_idens_{MODEL_TYPE}.pickle',
              'X_test': processed_path/f'final-test_x_{MODEL_TYPE}.pickle',
              'Y_test': processed_path/f'final-test_target_3_day_{MODEL_TYPE}.pickle',
              'idens_test': processed_path/f'final-test_idens_{MODEL_TYPE}.pickle'}

In [ ]:
DC = Datacard(client=CLIENT,
              data_paths=data_paths,
              report_conf_dir='.',
              report_output_dir='.',
              experiment_dates=EXPERIMENT_DATES)

In [ ]:
%%time
DC.generate_datacard()

## ===========================END======================================